In [15]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [3]:
class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [18]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC):
  @abstractmethod
  def iscount(self, order):
    pass

In [16]:
class Promotion(ABC):
  @abstractmethod
  def discount(self, order):
    pass

class Fidelitypromo(Promotion):
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemProme(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [7]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [19]:
Order(joe, cart, Fidelitypromo())

<Order total: 42.00 due: 42.00>

In [20]:
Order(ann, cart, Fidelitypromo())

<Order total: 42.00 due: 39.90>

In [22]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemProme())

<Order total: 30.00 due: 28.50>

In [24]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [26]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [27]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [28]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [29]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [30]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [31]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [32]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [35]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [36]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [37]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [38]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [39]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'park', '_2', '_i3', 'LineItem', '_i4', 'Order', '_i5', 'Promotion', '_i6', 'Fidelitypromo', 'BulkItemProme', 'LargeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_i8', '_i9', '_i10', '_i11', '_i12', '_i13', '_i14', '_14', '_i15', '_i16', '_i17', '_17', '_i18', '_i19', '_19', '_i20', '_20', '_i21', 'banana_cart', '_21', '_i22', '_22', '_i23', 'long_order', '_23', '_i24', '_24', '_i25', '_i26', '_26', '_i27', '_i28', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i29', '_29', '_i30', '_30', '_i31', '_31', '_i32', '_32', '_i33', 'promos', 'best_promo', '_i34', '_i35', '_i36', '_36', '_i37', '_37', '_i38', '_38', '_i39'])

In [41]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [43]:
promos = [globals() [name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [44]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [46]:
@deco
def target():
  print("running target()")
target()

running inner()


In [47]:
target

<function __main__.deco.<locals>.inner()>

In [50]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7a9c1a902cb0>)
running register(<function f2 at 0x7a9c1a902ef0>)
running main()
registry -> [<function f1 at 0x7a9c1a902cb0>, <function f2 at 0x7a9c1a902ef0>]
running f1()
running f2()
running f3()


In [51]:
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [52]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [53]:
b = 6
f1(3)

3
6


In [55]:
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [58]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [60]:
b

9

In [59]:
f3(3)

3
9


In [61]:
b= 30
f3(3)

3
30


In [63]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [64]:
avg = Averager()
avg(10)

10.0

In [65]:
avg(11)

10.5

In [66]:
avg(12)

11.0

In [67]:
def make_averger():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  return averager

In [68]:
avg = make_averger()
avg(10)

10.0

In [69]:
avg(11)

10.5

In [70]:
avg(12)

11.0

In [71]:
avg.__code__.co_varnames

('new_value', 'total')

In [72]:
avg.__code__.co_freevars

('series',)

In [73]:
avg.__closure__

(<cell at 0x7a9c1b14e230: list object at 0x7a9c1aa51500>,)

In [74]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [75]:
def make_averger():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  return averager

In [76]:
avg = make_averger()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [77]:
def make_averger():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager

In [79]:
avg = make_averger()
avg(10)

10.0

In [80]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [82]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316064s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000077s] factorial(1) -> 1
[0.00002864s] factorial(2) -> 2
[0.00004947s] factorial(3) -> 6
[0.00006966s] factorial(4) -> 24
[0.00009883s] factorial(5) -> 120
[0.00012062s] factorial(6) -> 720
6! = 720


In [85]:
factorial.__name__

'clocked'

In [86]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [88]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316561s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00003552s] factorial(2) -> 2
[0.00006080s] factorial(3) -> 6
[0.00010371s] factorial(4) -> 24
[0.00013232s] factorial(5) -> 120
[0.00016379s] factorial(6) -> 720
6! = 720


In [90]:
factorial.__name__

'factorial'